## Graph Analytics with ArangoDB (Python-Arango Library)

## Install Required Libraries

In [1]:
!pip3 install --upgrade pip

In [2]:
%pip install python-arango
%pip install networkx
%pip install numpy
%pip install scipy
%pip install tabulate
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Library Imports

In [3]:
import sys, csv, statistics

from arango import ArangoClient
import networkx as nx
import pkg_resources

import numpy as np
import scipy
from tabulate import tabulate
import geopandas as gpd

## Library Versions

In [4]:
l = 15
r = 14

arango_version = pkg_resources.get_distribution("python-arango").version

print("Software & Library Versions".center(l+r))
print('-'* (l + 1) + '|' + '-' * (r - 2))
print('Python'.rjust(l), '|', sys.version[0:6])
print('Arango Client'.rjust(l), '|', arango_version)
print('NetworkX'.rjust(l), '|', nx.__version__)
print('NumPy'.rjust(l), '|', np.__version__)
print('SciPy'.rjust(l), '|', scipy.__version__)

 Software & Library Versions 
----------------|------------
         Python | 3.11.4
  Arango Client | 8.1.2
       NetworkX | 3.4.2
          NumPy | 2.1.3
          SciPy | 1.14.1


## Import Dataset

In [5]:
# Connect to ArangoDB
client = ArangoClient()
db = client.db('_system', username='root', password='testpassword')

# Access collections
nodes = db.collection('airports')
edges = db.collection('flights')

# Fetch graph data
graph_data = {
    'nodes': list(nodes.all()),
    'edges': list(edges.all())
}

flightGraph = nx.MultiDiGraph()

# Add nodes
for node in graph_data['nodes']:
    flightGraph.add_node(node['_key'], **node)

# Add edges
for edge in graph_data['edges']:
    flightGraph.add_edge(
        edge['_from'].split('/')[-1],
        edge['_to'].split('/')[-1],
        **edge
    )

## Some Introductory Commands

### Show Number of Airports

In [6]:
query = """
FOR airport IN airports
    COLLECT WITH COUNT INTO numAirports
RETURN { NumberOfAirports: numAirports }
"""
result = db.aql.execute(query)
print(list(result))

[{'NumberOfAirports': 365}]


### Show the Number of Flights

In [7]:
query = """
FOR flight IN flights
    COLLECT WITH COUNT INTO numFlights
RETURN { NumberOfFlights: numFlights }
"""
result = db.aql.execute(query)

print(list(result))

[{'NumberOfFlights': 992298}]


### Add Airline Details for Each Airline Code (Relationship)

In [8]:
# Load CSV file
with open('../import/airlines.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        query = """
        FOR flight IN flights
            FILTER flight.airline_id == @airline_id
            UPDATE flight WITH {
                airline_name: @airline_name,
                airline_code: @airline_code
            } IN flights
        """
        db.aql.execute(query, bind_vars={
            'airline_id': row['airline_id'],
            'airline_name': row['airline_name'],
            'airline_code': row['airline_code']
        })

### Add Airport Name, City, & State For Each Airport (Node)

In [9]:
with open('../import/airports.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        query = """
        FOR airport IN airports
            FILTER airport.unique_id == @unique_id
            UPDATE airport WITH {
                airport_code: @airport_code,
                airport_name: @airport_name,
                airport_city: @city_name,
                airportState_code: @state,
                airportState_name: @state_name
            } IN airports
        """
        db.aql.execute(query, bind_vars={
            'unique_id': row['unique_id'],
            'airport_code': row['airport_code'],
            'airport_name': row['airport_name'],
            'city_name': row['city_name'],
            'state': row['state'],
            'state_name': row['state_name']
        })

## Utility Functions

#### Find Node ID Based On Value of Attribute

In [10]:
def convert_attr_value_to_node_id(graph, attribute_name, attribute_value):
    """
    Find the node in the graph that matches the given attribute value.
    
    Args:
        graph (nx.Graph): The NetworkX graph.
        attribute_name (str): The name of the attribute to search by.
        attribute_value (str): The value of the attribute to match.
        
    Returns:
        The ID of the node that matches the given attribute value, or None if not found.
    """
    for node, attributes in graph.nodes(data=True):
        if attributes.get(attribute_name) == attribute_value:
            return node # Return node ID
    return None

#### Find Value of Attribute Based On Node ID

In [11]:
def convert_node_id_to_attr_value(data, mapping_dict, attribute_name):
    """
    Replace keys in a list of tuples with the corresponding attribute values from a mapping dictionary.
    
    Args:
        data (list of tuples): The input list containing (key, value) pairs.
        mapping_dict (dict): A dictionary where keys are node/airport IDs and values are dictionaries of attributes.
        attribute_name (str): The name of the attribute to replace the key with.
        
    Returns:
        list: A list of tuples where the keys are replaced with the specified attribute values.
    """
    if not isinstance(data, list) or not all(isinstance(item, tuple) and len(item) == 2 for item in data):
        raise ValueError("Data must be a list of (key, value) tuples.")
    
    if not isinstance(mapping_dict, dict):
        raise ValueError("Mapping dictionary must be a valid dictionary.")
    
    if not isinstance(attribute_name, str):
        raise ValueError("Attribute name must be a string.")
    
    # Replace keys with the specified attribute value
    updated_data = []
    for key, value in data:
        attribute_value = mapping_dict.get(key, {}).get(attribute_name, key)  # Default is original key if attribute is missing
        updated_data.append((attribute_value, value))
    
    return updated_data

### Functions to Create Maps Between Ids & Attribute Values

In [12]:
# Function to create a map of node_id's as the keys and as the values

def create_airport_code_dict(G):
    """
    Extracts a dictionary mapping node IDs to their respective airport_code attributes from a NetworkX graph.
    
    Args:
        G (networkx.Graph): A NetworkX graph with nodes containing an 'airport_code' attribute.
        
    Returns:
        dict: A dictionary with node IDs as keys and airport_code as values.
    """
    # Initialize an empty dictionary
    airport_code_dict = {}

    # Iterate over nodes and their attributes
    for node, attributes in G.nodes(data=True):
        airport_code = attributes.get('airport_code')  # Get airport_code attribute
        if airport_code:  # Only include if airport_code attribute exists
            airport_code_dict[node] = airport_code  # Map node ID to airport_code

    return airport_code_dict

### --------------------------------------------------------------------
# Example output:

# {
#     1: "JFK",
#     2: "LAX"
# }
### --------------------------------------------------------------------

id_to_airport_code_mapping_dict = create_airport_code_dict(flightGraph)

print(id_to_airport_code_mapping_dict)
print(len(id_to_airport_code_mapping_dict))

{'179': 'FWA', '181': 'BZN', '183': 'RDM', '185': 'BKG', '187': 'MQT', '189': 'JAC', '191': 'MSP', '193': 'HIB', '195': 'UCA', '197': 'AUS', '199': 'HDN', '201': 'MAZ', '203': 'ANI', '205': 'CDC', '207': 'AMA', '209': 'PSC', '211': 'PVD', '213': 'AGS', '215': 'AVL', '217': 'CVG', '219': 'PLN', '221': 'PBI', '223': 'LMT', '225': 'BTM', '227': 'CPR', '229': 'ITH', '231': 'UTM', '233': 'CRW', '235': 'YUM', '237': 'DHN', '239': 'ATL', '241': 'MKG', '243': 'JAN', '245': 'LCH', '247': 'DIK', '249': 'LNY', '251': 'EWN', '253': 'ROW', '255': 'ITO', '257': 'ELM', '259': 'MWH', '261': 'HOB', '263': 'MCO', '265': 'EGE', '267': 'SLC', '269': 'CHO', '271': 'ECP', '273': 'ORD', '275': 'ISN', '277': 'SAN', '279': 'PDX', '281': 'GSO', '283': 'PIH', '285': 'MCN', '287': 'BNA', '289': 'SEA', '291': 'SAT', '293': 'SMX', '295': 'FSD', '297': 'BRD', '299': 'HOU', '301': 'EKO', '303': 'ERI', '305': 'GRR', '307': 'BFL', '309': 'SPN', '311': 'TUL', '313': 'INL', '315': 'DUT', '317': 'ATW', '319': 'HRL', '321'

### Convert Dictionary Keys From Node Id's to Airport Codes

In [13]:
# Convert each & every KEY from node_id to the airport_code value

def convert_output_keys_to_airport_codes(
    nx_output_dict, 
    node_to_airport_code_dict=id_to_airport_code_mapping_dict):
    """
    Converts the keys of a NetworkX output dictionary from node IDs to airport codes.
    
    Args:
        nx_output_dict (dict): A dictionary output from NetworkX with node IDs as keys.
        node_to_airport_code_dict (dict): A dictionary mapping node IDs to airport codes.
    
    Returns:
        dict: A dictionary with airport codes as keys and the original values preserved.
    """
    converted_dict = {}
    
    for node_id, value in nx_output_dict.items():
        # Get the airport code for the given node ID
        airport_code = node_to_airport_code_dict.get(
            node_id, 
            f"({node_id})"
            )
        
        # Add to the new dictionary
        converted_dict[airport_code] = value

    return converted_dict

# Example Use:
# converted_dict = convert_output_keys_to_airport_codes(nx_output_dict, id_to_airport_code_mapping_dict)

# Example Output:
# {
#     "JFK": 3,
#     "LAX": 5,
#     "ORD": 2
# }

In [14]:
# handle instances of the value being inputted as a single instance

def convert_single_instance(
    node_id, 
    node_to_airport_code_dict
    ):
    
    """
    Convert a single node_id (integer) to its corresponding airport_code.
    """
    return node_to_airport_code_dict.get(node_id, node_id)

In [15]:
# handle instances of the value being inputted as a list 

def convert_list(
    node_id_list, 
    node_to_airport_code_dict
    ):
    
    """
    Convert a list of node_ids (including nested lists) to their corresponding airport_codes.
    """
    converted_list = []
    for item in node_id_list:
        if isinstance(item, list):
            # Recursively process nested lists
            converted_list.append(convert_list(item, node_to_airport_code_dict))
        else:
            # Convert single elements
            converted_list.append(node_to_airport_code_dict.get(item, item))
    return converted_list

In [16]:
# handle instances of the value being inputted as a tuple 

def convert_tuple(
    node_id_tuple, 
    node_to_airport_code_dict
    ):
    
    """
    Convert a tuple of node_ids to their corresponding airport_codes.
    """
    return tuple(node_to_airport_code_dict.get(node_id, node_id) for node_id in node_id_tuple)

In [17]:
# handle instances of the value being inputted as a set

def convert_set(
    node_id_set, 
    node_to_airport_code_dict
    ):
    
    """
    Convert a set of node_ids to their corresponding airport_codes.
    """
    return {node_to_airport_code_dict.get(node_id, node_id) for node_id in node_id_set}

In [18]:
# handle instances of the value being inputted as a dictionary

def convert_dict(
    node_id_dict, 
    node_to_airport_code_dict
    ):
    
    """
    Convert a dictionary with node_ids in values to their corresponding airport_codes.
    """
    return {
        key: convert_values(
                value, 
                node_to_airport_code_dict
                ) 
        for key, value in node_id_dict.items()}

### Single Function to Handle All Dictionary Value Input Types

In [19]:
def convert_values(
    value, 
    node_to_airport_code_dict
    ):
    """
    Dynamically determine the type of input value and convert it appropriately.
    """
    if isinstance(value, int):
        return convert_single_instance(
            value, 
            node_to_airport_code_dict
            )
    elif isinstance(value, list):
        return convert_list(
            value, 
            node_to_airport_code_dict
            )
    elif isinstance(value, tuple):
        return convert_tuple(
            value, 
            node_to_airport_code_dict
            )
    elif isinstance(value, set):
        return convert_set(
            value, 
            node_to_airport_code_dict
            )
    elif isinstance(value, dict):
        return convert_dict(
            value, 
            node_to_airport_code_dict
            )
    else:
        # If value doesn't match any known type, return it as is
        return value

### Function(s) To Modify Output Format

#### Create Function to Return Statisics About Path Lengths (Dictionary Input)

In [20]:
def retrieve_path_stats(
    path_lengths
    ):
    
    """
    Analyze the values in a dictionary of path lengths.

    Parameters:
        path_lengths (dict): A dictionary where keys are nodes and values are path lengths.

    Returns:
        dict: A dictionary with max, mean, and count of the path lengths.
    """
    if not path_lengths:
        return {
            "min": None,
            "max": None, 
            "mean": None, 
            "count": 0,
            "std Dev": None,
            "harm_mean": None,
            "geo_mean": None,
            "quants": None,
            "mode": None
        }

    # Filter out non-numeric values from the list (e.g., lists, strings, etc.)
    values = [v for v in path_lengths.values() if isinstance(v, (int, float))]

    if not values: # Handle case where no valid numeric values exist
        return {
            "min": None,
            "max": None, 
            "mean": None, 
            "count": 0,
            "std Dev": None,
            "harmonic_mean": None,
            "geometric_mean": None,
            "quants": None,
            "mode": None
        }

    # Ensure all values are positive for geometric mean calculation
    positive_values = [v for v in values if v > 0]

    min_value = min(values)
    max_value = max(values)
    mean_value = statistics.mean(values)
    count = len(values)
    pop_st_dev = statistics.pstdev(values)  # This is the population standard deviation
    harm_mean = statistics.harmonic_mean(values)

    # Calculate the geometric mean if there are positive values
    if positive_values:
        geo_mean = statistics.geometric_mean(positive_values)
    else:
        geo_mean = None  # No valid positive values for geometric mean

    quants = statistics.quantiles(values, n=5)

    try:
        modes = statistics.mode(values)
    except statistics.StatisticsError:
        modes = None  # In case there are no unique modes

    return {
        "min": min_value,
        "max": max_value, 
        "mean": mean_value, 
        "count": count, 
        "std Dev": pop_st_dev,
        "harmonic_mean": harm_mean,
        "geometric_mean": geo_mean,
        "quants": quants,
        "mode": modes
    }

#### Function to Sort Dictionary in Order Based on Value & Return Top N Key/Value Pairs

In [21]:
def get_top_n_from_sorted_dict(
    input_dict, 
    n, 
    desc_order=True
    ):
    """
    Sort dictionary by its values in descending order & return top n key-value pairs.
    
    Args:
        input_dict (dict): The dictionary to sort.
        n (int): The number of top key-value pairs to return.
        desc (bool): True for sorting in descending order. False for ascending order.
        
    Returns:
        list: A list of tuples containing top n key-value pairs, sorted by value.
    """
    
    if not isinstance(input_dict, dict):
        raise ValueError("Input must be a dictionary.")
    if not isinstance(n, int) or n <= 0:
        raise ValueError("The number of top items (n) must be a positive integer.")
    if not isinstance(desc_order, bool) or n <= 0:
        raise ValueError("desc_order blue be a boolean value.")
    
    # Sort dictionary by values in descending order & extract top n items
    sorted_items = sorted(
        input_dict.items(), 
        key=lambda item: item[1], 
        reverse=desc_order
        )
    
    return sorted_items[:n]

### Other Utility Function(s)

#### Convert 'flight_distance' Attribute Values From String to Integer Values

In [22]:
# Convert 'flight_distance' attribute values from String to integer values
for u, v, d in flightGraph.edges(data=True):
    if 'flight_distance' in d:
        try:
            d['flight_distance'] = int(float(d['flight_distance']))  # Convert to float first, then integer
        except ValueError:
            print(f"Invalid flight_distance on edge ({u}, {v}): {d['flight_distance']}")
            d['flight_distance'] = 0  # Default value for invalid data

## Actual Analytics

#### Variables That Are Used Through Shortest Path Analysis

In [23]:
source = convert_attr_value_to_node_id(
    flightGraph, 
    "airport_code", 
    "MKE"
    )

target = convert_attr_value_to_node_id(
    flightGraph, 
    "airport_code", 
    "HLN"
    )

print("source:", source)
print("target:", target)

# Get airport_code values for source & target nodes using their id
source_airport_code = id_to_airport_code_mapping_dict.get(source, source)
target_airport_code = id_to_airport_code_mapping_dict.get(target, target)

print("source_airport_code:", source_airport_code)
print("target_airport_code:", target_airport_code)

source: 509
target: 417
source_airport_code: MKE
target_airport_code: HLN


### Shortest Paths (Unweighted Using Dijkstra's Algorithm)

In [24]:
method_used = 'dijkstra'

mke_hln_has_path = nx.has_path(
    flightGraph, 
    source, 
    target
    )

# Unweighted Dijkstra's Algorithms
dijk_unw_shortest_paths = convert_list(
    nx.shortest_path(
        flightGraph, 
        source, 
        target, 
        weight=None, 
        method=method_used
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_unw_all_shortest_paths = convert_list(
    [
        p for p in nx.all_shortest_paths(
            flightGraph, 
            source, 
            target, 
            weight=None, 
            method=method_used
            )
        ], 
    id_to_airport_code_mapping_dict
    )

dijk_unw_shortest_path_len = nx.shortest_path_length(
    flightGraph, 
    source, 
    target, 
    weight=None, 
    method=method_used
    )

dijk_unw_single_source = nx.single_source_shortest_path(
    flightGraph, 
    source, 
    cutoff=None
    )

dijk_unw_single_source = convert_output_keys_to_airport_codes(
    convert_values(
        dijk_unw_single_source, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_unw_single_source = {
    key: value for key, value in dijk_unw_single_source.items() 
        if key != source_airport_code
    } # removes source airport_code from results


dijk_unw_single_source_shortest_path_len = nx.single_source_shortest_path_length(
    flightGraph, 
    source, 
    cutoff=None
    )

dijk_unw_single_source_shortest_path_len = convert_output_keys_to_airport_codes(
    convert_values(
        dijk_unw_single_source_shortest_path_len, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_unw_single_source_shortest_path_len = {
    key: value for key, value in dijk_unw_single_source_shortest_path_len.items() 
        if key != source_airport_code
    } # removes source airport_code from results

dijk_unw_single_target_shortest_paths = nx.single_target_shortest_path(
    flightGraph, 
    target, 
    cutoff=None
    )

dijk_unw_single_target_shortest_paths = convert_output_keys_to_airport_codes(
    convert_values(
        dijk_unw_single_target_shortest_paths, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_unw_single_target_shortest_paths = {
    key: value for key, value in dijk_unw_single_target_shortest_paths.items() 
        if key != target_airport_code
    } # removes target airport_code from results

dijk_unw_single_target_shortest_paths_len = dict(
    nx.single_target_shortest_path_length(
        flightGraph, 
        target, 
        cutoff=None
        )
    )

dijk_unw_single_target_shortest_paths_len = convert_output_keys_to_airport_codes(
    convert_values(
        dijk_unw_single_target_shortest_paths_len, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_unw_single_target_shortest_paths_len = {
    key: value for key, value in dijk_unw_single_target_shortest_paths_len.items() 
        if key != target_airport_code
        } # removes the target airport_code from the results

# Other conversions that are required for printing
dijk_unw_single_source_shortest_path_len_rps = retrieve_path_stats(
    dijk_unw_single_source_shortest_path_len
    )

dijk_unw_single_target_shortest_paths_len_rps = retrieve_path_stats(
    dijk_unw_single_target_shortest_paths_len
    )

<class 'networkx.utils.decorators.argmap'> compilation 40:3: FutureWarning: 

single_target_shortest_path_length will return a dict instead of
an iterator in version 3.5


#### Print Results for Unweighted Dijkstra's Algorithm

In [25]:
# shortest paths for single source to single target
print("The following functions for this block/cell of code use Dijkstra's Algorithm & are unweighted.\n")
if not mke_hln_has_path:
    print(f"There is NOT a path between {source_airport_code} & {target_airport_code}.")
else:
    print(f"""There IS a path between {source_airport_code} & {target_airport_code}.
It is {dijk_unw_shortest_path_len} flights.
These are the airports (in order) for that flight path: {dijk_unw_shortest_paths}.
There is/are {len(dijk_unw_shortest_paths)} shortest paths, which are:
{dijk_unw_all_shortest_paths}.
""")

# Output for single source shortest paths

print(f"""Here is some data about the shortest paths between {source_airport_code} & other airports.
There are paths from {source_airport_code} to {len(dijk_unw_single_source)} other airports (not including itself).
Here are the descriptive statistics for flights from {source_airport_code}:
{dijk_unw_single_source_shortest_path_len_rps}
There is/are {len(dijk_unw_single_source)} shortest paths, which are:
{dijk_unw_single_source}.
""")

# output for the single target shortest paths
print(f"""Here is some data about the shortest paths between other airports & {target_airport_code}.
There are paths from {dijk_unw_single_target_shortest_paths_len} other airports to {target_airport_code} (not including itself).
Here are the descriptive statistics for flights arriving at {target_airport_code}:
{dijk_unw_single_target_shortest_paths_len_rps}
There is/are {len(dijk_unw_single_target_shortest_paths)} shortest paths, which are:
{dijk_unw_single_target_shortest_paths}.
""")

The following functions for this block/cell of code use Dijkstra's Algorithm & are unweighted.

There IS a path between MKE & HLN.
It is 2 flights.
These are the airports (in order) for that flight path: ['MKE', 'SLC', 'HLN'].
There is/are 3 shortest paths, which are:
[['MKE', 'MSP', 'HLN'], ['MKE', 'DEN', 'HLN'], ['MKE', 'SLC', 'HLN']].

Here is some data about the shortest paths between MKE & other airports.
There are paths from MKE to 349 other airports (not including itself).
Here are the descriptive statistics for flights from MKE:
{'min': 1, 'max': 4, 'mean': 1.9340974212034383, 'count': 349, 'std Dev': 0.4772300709850288, 'harmonic_mean': 1.7882152006831769, 'geometric_mean': 1.8673789437144497, 'quants': [2.0, 2.0, 2.0, 2.0], 'mode': 2}
There is/are 349 shortest paths, which are:
{'CVG': ['MKE', 'CVG'], 'ATL': ['MKE', 'ATL'], 'JFK': ['MKE', 'JFK'], 'MSP': ['MKE', 'MSP'], 'BOS': ['MKE', 'BOS'], 'DCA': ['MKE', 'DCA'], 'MCO': ['MKE', 'MCO'], 'TPA': ['MKE', 'TPA'], 'LGA': ['MKE', '

### Shortest Paths (Unweighted Using Bellman-Ford's Algorithm)

In [26]:
method_used = 'bellman-ford'

bf_unw_shortest_paths = convert_list(
    nx.shortest_path(
        flightGraph, 
        source, 
        target, 
        weight=None,
        method=method_used
        ), 
    id_to_airport_code_mapping_dict
    )

bf_unw_all_shortest_paths = convert_list(
    [
        p for p in nx.all_shortest_paths(
            flightGraph, 
            source, 
            target, 
            weight=None, 
            method=method_used
            )
        ], 
    id_to_airport_code_mapping_dict
    )

bf_unw_shortest_path_len = nx.shortest_path_length(
    flightGraph, 
    source, 
    target, 
    weight=None, 
    method=method_used
    )

bf_unw_single_source = nx.single_source_shortest_path(
    flightGraph, 
    source, 
    cutoff=None
    )

bf_unw_single_source = convert_output_keys_to_airport_codes(
    convert_values(
        bf_unw_single_source, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

bf_unw_single_source = {
    key: value for key, value in bf_unw_single_source.items() 
        if key != source_airport_code
    } # removes source airport_code from results

bf_unw_single_source_shortest_path_len = nx.single_source_shortest_path_length(
    flightGraph, 
    source, 
    cutoff=None
    )

bf_unw_single_source_shortest_path_len = convert_output_keys_to_airport_codes(
    convert_values(
        bf_unw_single_source_shortest_path_len, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

bf_unw_single_source_shortest_path_len = {
    key: value for key, value in bf_unw_single_source_shortest_path_len.items() 
        if key != source_airport_code
    } # removes source airport_code from results

bf_unw_single_target_shortest_paths = nx.single_target_shortest_path(
    flightGraph, 
    target, 
    cutoff=None
    )

bf_unw_single_target_shortest_paths = convert_output_keys_to_airport_codes(
    convert_values(
        bf_unw_single_target_shortest_paths, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

bf_unw_single_target_shortest_paths = {
    key: value for key, value in bf_unw_single_target_shortest_paths.items() 
        if key != target_airport_code
    } # removes target airport_code from results

bf_unw_single_target_shortest_paths_len = dict(
    nx.single_target_shortest_path_length(
        flightGraph, 
        target, 
        cutoff=None
        )
    )

bf_unw_single_target_shortest_paths_len = convert_output_keys_to_airport_codes(
    convert_values(
        bf_unw_single_target_shortest_paths_len, 
        id_to_airport_code_mapping_dict
        ), 
    id_to_airport_code_mapping_dict
    )

bf_unw_single_target_shortest_paths_len = {
    key: value for key, value in bf_unw_single_target_shortest_paths_len.items() 
        if key != target_airport_code
    } # removes target airport_code from results

# Other conversions that are required for the printing
bf_unw_single_source_shortest_path_len_rps = retrieve_path_stats(
    bf_unw_single_source_shortest_path_len
    )

bf_unw_single_target_shortest_paths_len_rps = retrieve_path_stats(
    bf_unw_single_target_shortest_paths_len
    )

In [27]:
# shortest paths for single source to single target
print("The following functions for this block/cell of code use Bellman-Ford's Algorithm & are unweighted.\n")
if not mke_hln_has_path:
    print(f"There is NOT a path between {source_airport_code} & {target_airport_code}.")
else:
    print(f"""There IS a path between {source_airport_code} & {target_airport_code}.
It is {bf_unw_shortest_path_len} flights.
These are the airports (in order) for that flight path: {bf_unw_shortest_paths}.
There is/are {len(bf_unw_shortest_paths)} shortest paths, which are: {bf_unw_all_shortest_paths}.
""")

# output for the single source shortest paths
print(f"""Here is some data about the shortest paths between {source_airport_code} & other airports.
There are paths from {source_airport_code} to {len(bf_unw_single_source)} other airports (not including itself).
Here are the descriptive statistics for flights from {source_airport_code}:
{bf_unw_single_source_shortest_path_len_rps}
There is/are {len(bf_unw_single_source)} shortest paths, which are: {bf_unw_single_source}.
""")

# output for the single target shortest paths
print(f"""Here is some data about the shortest paths between other airports & {target_airport_code}.
There are paths from {bf_unw_single_target_shortest_paths_len} other airports to {target_airport_code} (not including itself).
Here are the descriptive statistics for flights arriving at {target_airport_code}:
{bf_unw_single_target_shortest_paths_len_rps}
There is/are {len(bf_unw_single_target_shortest_paths)} shortest paths, which are: {bf_unw_single_target_shortest_paths}.
""")

The following functions for this block/cell of code use Bellman-Ford's Algorithm & are unweighted.

There IS a path between MKE & HLN.
It is 2 flights.
These are the airports (in order) for that flight path: ['MKE', 'SLC', 'HLN'].
There is/are 3 shortest paths, which are: [['MKE', 'MSP', 'HLN'], ['MKE', 'DEN', 'HLN'], ['MKE', 'SLC', 'HLN']].

Here is some data about the shortest paths between MKE & other airports.
There are paths from MKE to 349 other airports (not including itself).
Here are the descriptive statistics for flights from MKE:
{'min': 1, 'max': 4, 'mean': 1.9340974212034383, 'count': 349, 'std Dev': 0.4772300709850288, 'harmonic_mean': 1.7882152006831769, 'geometric_mean': 1.8673789437144497, 'quants': [2.0, 2.0, 2.0, 2.0], 'mode': 2}
There is/are 349 shortest paths, which are: {'CVG': ['MKE', 'CVG'], 'ATL': ['MKE', 'ATL'], 'JFK': ['MKE', 'JFK'], 'MSP': ['MKE', 'MSP'], 'BOS': ['MKE', 'BOS'], 'DCA': ['MKE', 'DCA'], 'MCO': ['MKE', 'MCO'], 'TPA': ['MKE', 'TPA'], 'LGA': ['MKE

#### Shortest Paths (flight_distance Weighted Using Dijkstra's Algorithm)

In [28]:
weighted_field = "flight_distance"
method_used = 'dijkstra'

### Dijkstra weighted algorithms
dijk_w_shortest_paths = convert_list(
    nx.shortest_path(
        flightGraph, 
        source, 
        target, 
        weight=weighted_field, 
        method=method_used
        ), 
    id_to_airport_code_mapping_dict
    )

dijk_w_all_shortest_paths = convert_list(
    [
        p for p in nx.all_shortest_paths(
            flightGraph, 
            source, 
            target, 
            weight=weighted_field, 
            method=method_used
            )
        ], 
    id_to_airport_code_mapping_dict
    )

dijk_w_shortest_path_len = nx.shortest_path_length(
    flightGraph, 
    source, 
    target, 
    weight=weighted_field, 
    method=method_used
    )

# Print Results
# shortest paths for single source to single target
print(f"""- The following functions for this block/cell of code use Dijkstra's Algorithm & are weighted using the {weighted_field} attribute.
- The shortest (weighted using the flight_distance attribute) path from {source_airport_code} to {target_airport_code} is {len(dijk_w_shortest_paths) - 1} flights 
    and the flight path (including the origin and destination airports) is: {dijk_w_shortest_paths}. 
- The flight distance of the shortest path is {dijk_w_shortest_path_len} miles.
- There is/are only {len(dijk_w_all_shortest_paths)} shortest flight patterns/paths between {source_airport_code} & {target_airport_code}.
""")

- The following functions for this block/cell of code use Dijkstra's Algorithm & are weighted using the flight_distance attribute.
- The shortest (weighted using the flight_distance attribute) path from MKE to HLN is 2 flights 
    and the flight path (including the origin and destination airports) is: ['MKE', 'MSP', 'HLN']. 
- The flight distance of the shortest path is 1210 miles.
- There is/are only 1 shortest flight patterns/paths between MKE & HLN.



#### Shortest Paths (flight_distance Weighted Using Bellman-Ford's Algorithm)

In [29]:
weighted_field = "flight_distance"
method_used = 'bellman-ford'

### Bellman Ford weighted algorithms
bf_w_shortest_paths = convert_list(
    nx.shortest_path(
        flightGraph, 
        source, 
        target, 
        weight=weighted_field, 
        method=method_used
        ), 
    id_to_airport_code_mapping_dict
    )

bf_w_all_shortest_paths = convert_list(
    [
        p for p in nx.all_shortest_paths(
            flightGraph, 
            source, 
            target, 
            weight=weighted_field, 
            method=method_used
            )
        ], 
    id_to_airport_code_mapping_dict
    )

bf_w_shortest_path_len = nx.shortest_path_length(
    flightGraph, 
    source, 
    target, 
    weight=weighted_field, 
    method=method_used
    )

# Print Results
# shortest paths for single source to single target
print(f"""- The following functions for this block/cell of code use Bellman-Ford's Algorithm & are weighted using the {weighted_field} attribute.
- The shortest (weighted using the flight_distance attribute) path from {source_airport_code} to {target_airport_code} is {len(bf_w_shortest_paths) - 1} 
    flights and the flight path (including the origin and destination airports) is: {bf_w_shortest_paths}. 
- The flight distance of the shortest path is {bf_w_shortest_path_len} miles.
- There is/are only {len(bf_w_all_shortest_paths)} shortest flight patterns/paths between {source_airport_code} & {target_airport_code}.
""")

- The following functions for this block/cell of code use Bellman-Ford's Algorithm & are weighted using the flight_distance attribute.
- The shortest (weighted using the flight_distance attribute) path from MKE to HLN is 2 
    flights and the flight path (including the origin and destination airports) is: ['MKE', 'MSP', 'HLN']. 
- The flight distance of the shortest path is 1210 miles.
- There is/are only 1 shortest flight patterns/paths between MKE & HLN.



#### s_metric
- s_metric

In [30]:
s_metric = nx.s_metric(flightGraph)

print(f"The S Metric valuefor this graph is {s_metric}")

The S Metric valuefor this graph is 1324914433835075.0


#### Structural Holes
- constraint
- effective_size
- local_constraint

In [35]:
source = convert_attr_value_to_node_id(
    flightGraph, 
    "airport_code",
    "MKE"
    )

target = convert_attr_value_to_node_id(
    flightGraph, 
    "airport_code", 
    "HLN"
    )

struct_hole_constraint = nx.constraint(flightGraph)

struct_hole_effective_size = nx.effective_size(flightGraph)

struct_hole_local_constraint = nx.local_constraint(
    flightGraph, 
    source, 
    target
    )

# Convert Resulting Dictionary Keys From Integers (ids) to String (airport codes)
struct_hole_constraint = convert_output_keys_to_airport_codes(
    struct_hole_constraint, 
    id_to_airport_code_mapping_dict
    )

struct_hole_effective_size = convert_output_keys_to_airport_codes(
    struct_hole_effective_size,
    id_to_airport_code_mapping_dict
    )

print("Struct Hole Constraint: ", struct_hole_constraint) # return both top & bottom 12 airport codes & their constraint values

struct_hole_constraint_top_n = sorted(struct_hole_constraint.items(), key=lambda x: x[1], reverse=True)[:12]

struct_hole_constraint_bottom_n = sorted(struct_hole_constraint.items(), key=lambda x: x[1], reverse=False)[:12]

print("Top 12 Struct Hole Constraint: ", struct_hole_constraint_top_n)
print("Bottom 12 Struct Hole Constraint: ", struct_hole_constraint_bottom_n)

print()
print("Struct Hole Effective Size", struct_hole_effective_size) # return both top & bottom 12 airport codes & their effective size values

struct_hole_effective_size_top_n = sorted(
    struct_hole_effective_size.items(), 
    key=lambda x: x[1], 
    reverse=True
    )[:12]

struct_hole_effective_size_bottom_n = sorted(
    struct_hole_effective_size.items(), 
    key=lambda x: x[1], 
    reverse=False
    )[:12]

print("Top 12 Struct Hole Effective Sizes: ", struct_hole_effective_size_top_n)
print("Bottom 12 Struct Hole Effective Sizes: ", struct_hole_effective_size_bottom_n)
print()
print("Struct Hole Local Constraint: ", struct_hole_local_constraint)

Struct Hole Constraint:  {'FWA': 0.16539374788845443, 'BZN': 0.11130580625301997, 'RDM': 0.21879024961206556, 'BKG': 0.1085620989963449, 'MQT': 0.2352561267552961, 'JAC': 0.11899272447976846, 'MSP': 0.0346568064793995, 'HIB': 0.698161865569273, 'UCA': nan, 'AUS': 0.04970018629717188, 'HDN': 0.10745229126473382, 'MAZ': 1.0, 'ANI': 0.820018378257964, 'CDC': 0.6961446619936278, 'AMA': 0.16355305186715255, 'PSC': 0.2020523792640574, 'PVD': 0.059167165518603765, 'AGS': 0.19583571171732778, 'AVL': 0.09767644799853284, 'CVG': 0.03939144510471137, 'PLN': 0.6981480977171395, 'PBI': 0.06001622551721327, 'LMT': 0.5149723230255285, 'BTM': 1.0, 'CPR': 0.5085006302943753, 'ITH': 0.46751433753813254, 'UTM': 1.0, 'CRW': 0.09917189620543568, 'YUM': 0.42807977862164415, 'DHN': 1.0, 'ATL': 0.03110697919341483, 'MKG': 1.0, 'JAN': 0.09210724367996573, 'LCH': 0.5068326558765741, 'DIK': 0.5071420387674336, 'LNY': 0.5, 'EWN': 1.0, 'ROW': 0.3462573707881752, 'ITO': 0.2716908439244813, 'ELM': 0.356892807901228,